In [12]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [13]:
orders = pd.read_csv('Orders_all_clean.csv', index_col=None, engine='python')
orders.head()

,Contact_Email_Address,Created,Account_SIC,Price_List,Price_List_Type,Total_Net_Price,Account_Market,Account_Name,Campaign_Name,Contact_First_Name,...,Event_Plan_Code,Event_Plan_Name,Event_Plan_Start_Date,MAC,Order_Date,Order_Number,Registration_Channel,Registration_Type,Sales_Method,Sales_Source
0,jfaddis@taleo.com,1/3/11 9:22,Management Consulting Services,19698.001 - Vendor,Vendor,"$15,500.00",Secondary,Taleo,NaN,Josh,...,19698.001,HCMF Federal Recruitment Strategy Day,5/3/11 20:00,NaN,1/3/11,1-3341865771,NaN,Sponsor,NaN,SMBARRETT
1,christina.morgan1@us.army.mil,1/3/11 9:47,Army,10665.005 - Standard,Primary,"$1,949.00",Primary,US Army Audit Agency,10665.005 HCMD 2011: Unknown without MACs,Christina,...,10665.005,HCMD 2011,2/13/11 19:00,NaN,1/3/11,1-3341957177-A,Website,Delegate,NaN,NaN
2,tfreeth@google.com,1/3/11 9:48,Web Search Portals,10701.005-Vendor,Vendor,"$3,089.00",Secondary,Google,10701.005 ETail West 2011 Delegate Sales Campa...,Tim,...,10701.005,eTail 2011 West,2/21/11 19:00,1-3274993282,1/3/11,1-3342074626,Outbound Phone,Delegate,Full Delegate Sale,TSPETERS
3,charley@gigya-inc.com,1/3/11 9:51,Software As A Service Solution Providers,10701.005-Vendor,Vendor,$0.00,Secondary,Gigya,NaN,Charley,...,10701.005,eTail 2011 West,2/21/11 19:00,NaN,1/3/11,1-3342079261,Phone,Sponsorship Guest,NaN,NaN
4,autumn.coleman@servigistics.com,1/3/11 9:53,Software As A Service Solution Providers,10553.005 - Vendor,Vendor,$0.00,Secondary,PTC,NaN,Autumn,...,10553.005,Interlog Winter 2011,1/24/11 19:00,NaN,1/3/11,1-3342070411-A,NaN,Sponsorship Guest,NaN,NaN


In [15]:
# remove the negative price orders either mistake or cancellation and currency symbols
orders['Total_Net_Price'] = orders['Total_Net_Price'].replace('[\$, ]', '', regex=True)  \
                           .replace('[\€, ]', '', regex=True).replace('[\�, ]', '', regex=True)  \
                           .replace('[R, ]', '', regex=True)
orders = orders[orders.Total_Net_Price != '(26.00)']
orders['Registration_Channel'] = orders['Registration_Channel'].fillna(value='Not Available').astype('category')

# Attributing sales source by registration type, creating new categorical series.
orders.loc[orders.Registration_Type == 'Speaker Guest', 'Sales_Source'] = 'Production'
orders.loc[orders.Registration_Type == 'Speaker', 'Sales_Source'] = 'Production'
orders.loc[orders.Registration_Type == 'Sponsor Attendee', 'Sales_Source'] = 'Sponsorship'
orders.Sales_Source.fillna(value='Marketing', inplace=True)
orders['Sales_Source_Cat'] = pd.Series()
orders.loc[orders.Registration_Type == 'Speaker Guest', 'Sales_Source_Cat'] = 'Production'
orders.loc[orders.Registration_Type == 'Speaker', 'Sales_Source_Cat'] = 'Production'
orders.loc[orders.Registration_Type == 'Sponsor Attendee', 'Sales_Source_Cat'] = 'Sponsorship'
orders.loc[orders.Registration_Type == 'Sponsor', 'Sales_Source_Cat'] = 'Sponsorship'
orders.loc[orders.Sales_Source == 'Marketing', 'Sales_Source_Cat'] = 'Marketing'
orders.Sales_Source_Cat.fillna(value='Delegate Sales', inplace=True)

In [16]:
# Creating unnique IDs for each person pased on first name, last name and email
# Original columns will be removed for data propection
orders['id'] = orders.groupby(['Contact_First_Name','Contact_Last_Name', 'Contact_Email_Address']).ngroup()

In [17]:
orders.drop(columns=['Contact_First_Name','Contact_Last_Name', 'Contact_Email_Address'])

,Created,Account_SIC,Price_List,Price_List_Type,Total_Net_Price,Account_Market,Account_Name,Campaign_Name,Contact_Job_Function,Contact_Job_Title,...,Event_Plan_Start_Date,MAC,Order_Date,Order_Number,Registration_Channel,Registration_Type,Sales_Method,Sales_Source,Sales_Source_Cat,id
0,1/3/11 9:22,Management Consulting Services,19698.001 - Vendor,Vendor,15500.00,Secondary,Taleo,NaN,LW10,Senior Vice President and General Counsel,...,5/3/11 20:00,NaN,1/3/11,1-3341865771,Not Available,Sponsor,NaN,SMBARRETT,Sponsorship,34473
1,1/3/11 9:47,Army,10665.005 - Standard,Primary,1949.00,Primary,US Army Audit Agency,10665.005 HCMD 2011: Unknown without MACs,HR254,Human Capital Manager,...,2/13/11 19:00,NaN,1/3/11,1-3341957177-A,Website,Delegate,NaN,Marketing,Marketing,12735
2,1/3/11 9:48,Web Search Portals,10701.005-Vendor,Vendor,3089.00,Secondary,Google,10701.005 ETail West 2011 Delegate Sales Campa...,HR253,Head Of Industry,...,2/21/11 19:00,1-3274993282,1/3/11,1-3342074626,Outbound Phone,Delegate,Full Delegate Sale,TSPETERS,Delegate Sales,66732
3,1/3/11 9:51,Software As A Service Solution Providers,10701.005-Vendor,Vendor,0.00,Secondary,Gigya,NaN,MK48,National Sales Director,...,2/21/11 19:00,NaN,1/3/11,1-3342079261,Phone,Sponsorship Guest,NaN,Marketing,Marketing,11220
4,1/3/11 9:53,Software As A Service Solution Providers,10553.005 - Vendor,Vendor,0.00,Secondary,PTC,NaN,MK7,Marketing Communications Manager,...,1/24/11 19:00,NaN,1/3/11,1-3342070411-A,Not Available,Sponsorship Guest,NaN,Marketing,Marketing,5795
5,1/3/11 9:56,Software As A Service Solution Providers,10701.005-Vendor,Vendor,0.00,Secondary,Gigya,NaN,MG1,Chief Executive Officer,...,2/21/11 19:00,NaN,1/3/11,1-3342079663,Not Available,Sponsorship Guest,NaN,Marketing,Marketing,52975
6,1/3/11 9:57,Electronic Shopping,10701.005-Standard,Primary,1259.35,Primary,PriceMatters.ca,10701.005 eTail West 2011 Marketing Unknowns W...,IT294,E-Commerce Manager,...,2/21/11 19:00,NaN,1/3/11,1-3342074956,Website,Delegate,NaN,Marketing,Marketing,70764
7,1/3/11 9:57,National Security,10665.005 - Standard,Primary,1499.00,Primary,Army Audit Agency,10665.005 HCMD 2011: Unknown without MACs,UU100,Strategy Human Capital Advisor,...,2/13/11 19:00,NaN,1/3/11,1-3341957177-B,Website,Delegate,NaN,Marketing,Marketing,12517
8,1/3/11 9:59,Software As A Service Solution Providers,10553.005 - Vendor,Vendor,0.00,Secondary,PTC,NaN,MK48,"Sales Business Transformation Director, Fellow",...,1/24/11 19:00,NaN,1/3/11,1-3342070411-C,Not Available,Speaker,NaN,Production,Production,19693
9,1/3/11 10:04,Software As A Service Solution Providers,10701.005-Vendor,Vendor,1529.00,Secondary,Symantec,10701.005 eTail West 2011 DLS Unknowns Without...,MK7,Global Marketing Strategy Manager,...,2/21/11 19:00,NaN,1/3/11,1-3342088336,Outbound Phone,Delegate,PDF Conversion,TSNEVEU,Delegate Sales,4347


In [19]:
# Creating flags for group registrations - Group size variable indicates number in group of attendees

orders = orders[orders.Order_Number.notnull()]
orders['Order_Number']=orders['Order_Number'].astype(str)
orders['Group_Size'] = ""
dict = {}

for i in orders['Order_Number']:
    x = i[:12]
    if not x in dict:
        dict[x] = 1
    else:
        dict[x] += 1

group_size = []

for i in orders['Order_Number']:
    x = i[:12]
    group_size.append(dict[x])

orders['Group_Size'] = group_size

orders['Order_Number'].groupby(orders['Group_Size']).agg('count')

Group_Size
1     74573
2     13834
3      6864
4      3952
5      1930
6       996
7       476
8       472
9       234
10      260
11      132
12      144
13       91
14       14
15       30
17       17
20       20
21       42
Name: Order_Number, dtype: int64

In [20]:
#Creating a list of job title seniority based on full job titles. Appending list to DataFrame

orders['Contact_Job_Title'] = orders.Contact_Job_Title.fillna("UNKNOWN")

job_title = []
c_level = ['Chief','CEO','CMO','CDO','CIO','CPO','CTO','Head']
vp = ['VP','Vice President']

for i in orders['Contact_Job_Title']:
    if any(title in i for title in c_level):
        job_title.append("C-Level")
    elif any(title in i for title in vp):
        job_title.append("VP")
    elif "Director" in i:
        job_title.append("Director")
    elif "Manager" in i:
        job_title.append("Manager")
    elif "Consultant" in i:
        job_title.append("Consultant")
    else:
        job_title.append("Other")
        
orders['job_title'] = job_title
orders['job_title'].value_counts()

Director      25797
Manager       24560
Other         24074
VP            16218
C-Level       12349
Consultant     1083
Name: job_title, dtype: int64

In [21]:
orders.to_csv('orders_after_wrangling.csv')